#Download
Download 3 things that saved during weibo.ipynb:
* Dictionaries for One-hot encoding in Embeddinng layers.  
* metrics' means and standard deviations (for input normalization). Because the data is normalized by $(data-mean)/std$ before training the model, the same means and stds have to be used to normalize the new dataset.  
* model and its weights

P.S. For those who understand Chinese language or use weibo, you can create a csv file with one column named "uid", and then fill in the user ids to be detected. Please save it as "uids_for_detection.csv" and upload it.


In [ ]:
import pandas as pd
!gdown https://drive.google.com/uc?id=1nbWJO9wGMODfnmYGCASi9kEozpgtIRjo
!unzip weibo_baselines.zip

Downloading...
From: https://drive.google.com/uc?id=1nbWJO9wGMODfnmYGCASi9kEozpgtIRjo
To: /content/weibo_baselines.zip
38.9MB [00:00, 106MB/s] 
Archive:  weibo_baselines.zip
   creating: weibo0.15split_testAcc90.48%/
   creating: weibo0.50split_testAcc90.14%/
   creating: weibo0.85split_testAcc98.84%/
  inflating: uids_for_detection.csv  
  inflating: weibo0.50split_testAcc90.14%/metrics_means_and_stds.csv  
  inflating: weibo0.50split_testAcc90.14%/weibo_model.h5  
  inflating: weibo0.50split_testAcc90.14%/token2idW.csv  
  inflating: weibo0.50split_testAcc90.14%/token2idTR.csv  
  inflating: weibo0.50split_testAcc90.14%/weibo_dataset.csv  
   creating: weibo0.85split_testAcc98.84%/.ipynb_checkpoints/
  inflating: weibo0.85split_testAcc98.84%/metrics_means_and_stds.csv  
  inflating: weibo0.85split_testAcc98.84%/weibo_model.h5  
  inflating: weibo0.85split_testAcc98.84%/token2idW.csv  
  inflating: weibo0.85split_testAcc98.84%/token2idTR.csv  
  inflating: weibo0.85split_testAcc98.84%

In [ ]:
file_name='uids_for_detection.csv'
uid_label_dataset=pd.read_csv(file_name,sep=',')

#Crawler, crawl the weibo information corresponding to uids

In [ ]:
%pylab inline
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output
dataset_csv=pd.DataFrame()
index=0
while index< len(uid_label_dataset):
  row=uid_label_dataset.iloc[index]
  print("Crawling the %s th user"%(index+1))
  try:
    userid=str(row[0])
    headers = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}
    def get_json(param):
      return requests.get('https://m.weibo.cn/api/container/getIndex?containerid='+param,headers=headers,verify=False).json()
    js = get_json('100505'+userid)
    info_keys=['follow_count', 'followers_count', 'mbrank', 'mbtype', 'statuses_count', 'urank', 'verified', 'verified_type', 'verified_type_ext']
    if js['ok']:
      info={k:v for k,v in js['data']['userInfo'].items() if k in info_keys }
      if not info.get('verified_type_ext'):info['verified_type_ext']=-1
      fans_interact=get_json('231051_-_fans_intimacy_-_'+userid)['data']['cards']
      info['interact']=-1
      if js['ok']:
        if any(["没有" in f.get('desc',"") for f in fans_interact]):
          info['interact']=0
        else:
          info['interact']=sum([int(f.get('desc1').split()[1][3:]) for f in fans_interact[1]['card_group']])
      page=1
      js=get_json('107603' + userid+'&count='+str(30)+'&page='+str(page))
      if js['ok']:
        weibo_infos,topics,retwefeatuers,weibos,num_pics,video_play_counts,reposts_counts,comments_counts,attitudes_counts=[],[],[],[],[],[],[],[],[]
        for i in range(len(js['data']['cards'])):
          weibos.append('\n'.join([l for l in list(BeautifulSoup(js['data']['cards'][i]['mblog']['text'],'lxml').stripped_strings) if "@" not in l and "#" not in l]))
          num_pics.append(len(js['data']['cards'][i]['mblog'].get('pic_ids',[])))
          video_play_counts.append(int(js['data']['cards'][i]['mblog'].get('page_info',{"play_count":"-1次播放"}).get('play_count',"-1次播放")[:-3].replace("万","0000").replace("亿","00000000")))
          reposts_counts.append(int(str(js['data']['cards'][i]['mblog']['reposts_count']).replace("万","0000").replace("亿","00000000").replace("+","")))
          comments_counts.append(int(str(js['data']['cards'][i]['mblog']['comments_count']).replace("万","0000").replace("亿","00000000").replace("+","")))
          attitudes_counts.append(int(str(js['data']['cards'][i]['mblog']['attitudes_count']).replace("万","0000").replace("亿","00000000").replace("+","")))
          weibo_infos.append([num_pics[i],video_play_counts[i],reposts_counts[i],comments_counts[i],attitudes_counts[i]])
          topics.append('\n'.join([l for l in list(BeautifulSoup(js['data']['cards'][i]['mblog']['text'],'lxml').stripped_strings) if  "#"  in l]))
          retwefeatuers.append('\n'.join([l for l in list(BeautifulSoup(js['data']['cards'][i]['mblog'].get('retweeted_status',{"text":"noretweet"}).get("text"),'lxml').stripped_strings) if "#"in l or "视频" in l or "noretweet" in l]))
        info['n_weibos']=str(weibos)
        info['n_topics']=str(topics)
        info['n_weibo_infos']=str(weibo_infos)
        info['n_retwefeatuers']=str(retwefeatuers)
        info['uid']=int(row[0])
        dataset_csv=dataset_csv.append(info, ignore_index=True)
    index+=1
    if index%20==0:
      dataset_csv.fillna(-1,inplace=True)
      dataset_csv.to_csv('weibo_dataset.csv', index=False,encoding="utf_8_sig")
    clear_output()
  except Exception as e:
    print(e)
    time.sleep(5+np.random.rand()*3)
    pass
print("The crawling of "+str(index)+" users has ended (except for some users who have been blocked) and has been saved to \"weibo_dataset.csv\"")
dataset_csv.fillna(-1,inplace=True)
dataset_csv.to_csv('weibo_dataset.csv', index=False,encoding="utf_8_sig") 

The crawling of 2 users has ended (except for some users who have been blocked) and has been saved to "weibo_dataset.csv"


#Load framework and TPU environment

In [ ]:
%pylab inline
import tensorflow as tf
import pandas as pd
import jieba
from gensim import corpora
from keras.preprocessing import sequence
from scipy import stats
import ast
import os
import keras
from keras.models import Model,Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Reshape,Input, LSTM,Bidirectional,GRU, concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['info']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


INFO:tensorflow:Initializing the TPU system: grpc://10.32.137.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.32.137.130:8470


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


#Load the model and its weights
I choose the one trained with 85% data set.

In [ ]:
cd weibo0.85split_testAcc98.84%/

/content/weibo0.85split_testAcc98.84%


In [ ]:
with strategy.scope():
  model = tf.keras.models.load_model("weibo_model.h5")
maxlens=[i[1] for i in model.input_shape[0]][:2]

#Load metrics' means and standard deviations

In [ ]:
metrics_means_and_stds=pd.read_csv('metrics_means_and_stds.csv')
wmd_mean=ast.literal_eval(metrics_means_and_stds['weibo_metrics_dataset_mean'][0])
wmd_std=ast.literal_eval(metrics_means_and_stds['weibo_metrics_dataset_std'][0])
umd_mean=ast.literal_eval(metrics_means_and_stds['user_metrics_dataset_mean'][0])
umd_std=ast.literal_eval(metrics_means_and_stds['user_metrics_dataset_std'][0])

#Read the test set from csv file and convert it to the input format of the model

In [ ]:
maxlens=[i[1] for i in model.input_shape[0]]
num_weibo=len(model.input_shape)-1
testset_dataset=pd.read_csv('../weibo_dataset.csv',keep_default_na=False)
dicW=pd.read_csv('token2idW.csv',keep_default_na=False)
dicW={k:v for k,v in dicW.values}
dicTR=pd.read_csv('token2idTR.csv',keep_default_na=False)
dicTR={k:v for k,v in dicTR.values}
n_weibos=testset_dataset['n_weibos'].values
def chinese_cut_token(sentences,dic=None):
  sentences_cut=[]
  for input in sentences:
    input=ast.literal_eval(input)
    sentence_cut=[["None"]]*num_weibo
    for i,weibo in enumerate(input):
      if (i>=num_weibo): break
      tokens=jieba.cut(weibo, cut_all=True)
      sentence_cut[i]=[word for word in tokens]
    sentences_cut.append(sentence_cut[:num_weibo])
  dic=corpora.Dictionary([token for sentence_cut in sentences_cut for token in sentence_cut]).token2id if dic==None else dic
  X=[]
  for user in sentences_cut:
    x=[]
    for weibo_sentence in user:
      x.append([dic.get(token,-1)+2 for token in weibo_sentence])
    X.append(x)
  maxLen=max(len(max(x,key=len)) for x in X)
  wordsSize =len(dic)+1
  return sentences_cut,dic,array(X),maxLen,wordsSize
_,_,XW,_,_=chinese_cut_token(n_weibos,dicW)
maxLenW=maxlens[0]
n_weibos_seq2 = sequence.pad_sequences(XW.reshape(-1), maxLenW).reshape(len(n_weibos),num_weibo,maxLenW)
n_topicsAndRetwefeatuers2=append(testset_dataset['n_retwefeatuers'],testset_dataset['n_topics'])
_,_,XTR,_,_=chinese_cut_token(n_topicsAndRetwefeatuers2,dicTR)
maxLenTR=maxlens[1]
n_topicsAndRetwefeatuers_seq2 = sequence.pad_sequences(XTR.reshape(-1), maxLenTR).reshape(len(n_topicsAndRetwefeatuers2),num_weibo,maxLenTR)
n_retwefeatuers_seq2=n_topicsAndRetwefeatuers_seq2[:len(testset_dataset['n_retwefeatuers'])]
n_topics_seq2=n_topicsAndRetwefeatuers_seq2[len(testset_dataset['n_retwefeatuers']):]
user_metrics_dataset2=testset_dataset[['follow_count', 'followers_count', 'mbrank', 'mbtype', 'statuses_count', 'urank', 'verified', 'verified_type', 'verified_type_ext']]
user_metrics_dataset2=(user_metrics_dataset2-umd_mean)/umd_std
tmp=[ast.literal_eval(row) for row in testset_dataset['n_weibo_infos']]
weibo_metrics_dataset2=ones((len(tmp),num_weibo,len(tmp[0][0])))*-2
for i in range(weibo_metrics_dataset2.shape[0]):
  weibo_metrics_dataset2[i,:len(tmp[i])]=tmp[i][:num_weibo]
weibo_metrics_dataset2=array([(wi-wmd_mean)/wmd_std for wi in weibo_metrics_dataset2])
testInputs=[*[(n_weibos_seq2[:,i],n_retwefeatuers_seq2[:,i],n_topics_seq2[:,i],weibo_metrics_dataset2[:,i])for i in range(num_weibo)],user_metrics_dataset2]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.967 seconds.
DEBUG:jieba:Loading model cost 0.967 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


#Whether you know Chinese or not, please check these two links, then check the predicted result.
https://m.weibo.cn/u/2214257545  
https://m.weibo.cn/u/1847677113  
Result will be saved in "weibo_detection_result.csv" 


In [ ]:
pred=model.predict(testInputs).argmax(axis=1)
output_csv=pd.DataFrame(columns=['uid','bot or not'])
output_csv['uid']=testset_dataset['uid'].astype(int)
output_csv['bot or not']=pred.astype(bool)
output_csv.to_csv('../weibo_detection_result.csv',index=False,encoding="utf_8_sig")
output_csv

,uid,bot or not
0,2214257545,False
1,1847677113,False
